In [1]:
!pip install -U bitsandbytes
!pip install -U accelerate transformers
!pip install openai-whisper


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 126.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 18.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=d0d664aca4b7f6c30b74f871fbec884b864ed2618003892448eba9d3f9fc982f
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [3]:
import whisper
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Options: "tiny", "base", "small", "medium", "large-v3"
model = whisper.load_model("tiny", device=device)

audio_path = "/content/spanish_audio.wav"
result = model.transcribe(audio_path)
transcript = result['text']
print("Transcription:\n", result['text'])
print("Language:\n", result['language'])


Using device: cuda
Transcription:
  La inteligencia artificial está transformando el mundo.
Language:
 es


In [4]:
print(transcript)

 La inteligencia artificial está transformando el mundo.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login


# Load Mistral-7B-Instruct
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

HF_TOKEN = "random_token"  
login(HF_TOKEN)

bnb_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    use_auth_token=HF_TOKEN
)

prompt_translate = f"Translate this to English:\n{transcript}\n\nOnly output the translation."
inputs = tokenizer(prompt_translate, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=80, eos_token_id=tokenizer.eos_token_id)
translation = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().split("\n")[-1]

print("Translation:", translation)

prompt_rephrase = f"Rephrase this sentence in casual English:\n{translation}\n\nOnly output the rephrased sentence."
inputs = tokenizer(prompt_rephrase, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=80, eos_token_id=tokenizer.eos_token_id)
rephrased = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().split("\n")[-1]

print("Rephrased:", rephrased)


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Translation: Artificial intelligence is transforming the world.
Rephrased: AI is really changing things up in the world.
